In [ ]:
import os
import pandas as pd
import csv
import theano
import theano.tensor as T
import lasagne
import numpy as np
import sklearn.datasets
from recurrent import gen_data
from keras.models import Sequential
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.optimizers import RMSprop
from keras.regularizers import l2, activity_l2
%pylab inline

In [ ]:
curr_dir = os.path.dirname('__file__')
file_path = os.path.join(curr_dir, "temps.csv")

In [ ]:
with open(file_path, 'rb') as csvfile:
    reader = csv.reader(csvfile)
    data = list(reader)

data = data[1:-3]
for row in data:
    row[1] = float(row[1].replace("?", ""))
df = pd.DataFrame(data, columns=['Date', 'Temperature'])
df = pd.DataFrame(df.Temperature)

In [ ]:
def create_time_series_epoch(data, ravel=False, n_prev=14):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)
    if (ravel):
        alsY = alsY.ravel()

    return alsX, alsY

def train_test_split(df, test_size=0.1):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))
    
    X_train, y_train = create_time_series_epoch(df.iloc[0:ntrn])
    X_test, y_test = create_time_series_epoch(df.iloc[ntrn:])

    x_mean = np.mean(X_train)
    x_std = np.std(X_train)
    y_mean = np.mean(y_train)
    y_std = np.std(y_train)

    return (X_train - x_mean, y_train - y_mean), (X_test - x_mean, y_test - y_mean)

In [ ]:
(X_train, y_train), (X_test, y_test) = train_test_split(df)  # retrieve data

In [ ]:
in_out_neurons = 1  
hidden_neurons = 10

l2_possibilities_1 = np.logspace(-1, 1, 3)
l2_possibilities_2 = np.logspace(-2, 0, 3)

best_model = None
best_score = 1000

for val_1 in l2_possibilities_1:
    for val_2 in l2_possibilities_2:
        model = Sequential()
        model.add(LSTM(hidden_neurons, input_dim=in_out_neurons, W_regularizer=l2(val_1), activation='tanh'))
        model.add(Dense(1, W_regularizer=l2(val_2)))
        model.compile(loss="mean_squared_error", optimizer=RMSprop(lr=.01))
        model.fit(X_train, y_train, nb_epoch=10, verbose=0)
        predicted = model.predict(X_test)
        rmse = ((predicted - y_test) ** 2).mean(axis=0)
        if (rmse < best_score):
            best_score = rmse
            best_model = model
        print "RMSE for l2 of %f, %f: %f" % (val_1, val_2, rmse)

In [ ]:
predicted = best_model.predict(X_test)
print "Error on train set: %.1f" % ((best_model.predict(X_train) - y_train) ** 2).mean()
print "Error on test set: %.1f" % ((best_model.predict(X_test) - y_test) ** 2).mean()
print "Previous day error: %.1f" % ((X_test[:, -1, :] - y_test) ** 2).mean()

In [ ]:
predicted = best_model.predict(X_test)
plot(pd.DataFrame(predicted)[40:60])
plot(pd.DataFrame(y_test)[40:60])
# plot(X_test[:, -1, :][40:60])
legend(['pred', 'actual'])